In [5]:
import torch
import uproot
import random
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import lightning as L

In [6]:
# basic random seed
def seed_basic(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)


# torch random seed
def seed_torch(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# basic + torch + lightning
def seed_everything(seed=42):
    seed_basic(seed)
    seed_torch(seed)
    L.seed_everything(seed)


seed_everything()

Seed set to 42


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
file_sm = uproot.open("../data/SM.root")
file = uproot.open("../data/DM.root")

df_DM = file['LHE'].arrays(file['LHE'].keys(), library="pd")
df_SM = file_sm['LHE'].arrays(file_sm['LHE'].keys(), library="pd")
df_SM['M_phi'] = np.sqrt(df_SM['E_phi']**2 - (df_SM['p_phi_x']**2 + df_SM['p_phi_y']**2 + df_SM['p_phi_z']**2))
df_DM['M_phi'] = np.sqrt(df_DM['E_phi']**2 - (df_DM['p_phi_x']**2 + df_DM['p_phi_y']**2 + df_DM['p_phi_z']**2))

train_raw_SM, test_raw_SM = train_test_split(df_SM, test_size=0.2)
train_raw_SM, val_raw_SM = train_test_split(train_raw_SM, test_size=0.2)

train_raw_DM, test_raw_DM = train_test_split(df_DM, test_size=0.2)
train_raw_DM, val_raw_DM = train_test_split(train_raw_DM, test_size=0.2)

train_raw = pd.concat([train_raw_SM, train_raw_DM], axis=0) # We use the 1/1 mix of SM and DM datasets for training
val_raw = pd.concat([val_raw_SM, val_raw_DM], axis=0)
test_raw = pd.concat([test_raw_SM, test_raw_DM], axis=0)

In [9]:
y_vars = ['p_nu_x', 'p_nu_y', 'p_nu_z', 'p_phi_x', 'p_phi_y', 'p_phi_z']
X_vars = ['p_l_x', 'p_l_y', 'p_l_z', 'p_b_x', 'p_b_y', 'p_b_z', 'p_q_x', 'p_q_y',
          'p_q_z', 'MET_x', 'MET_y', 'MET_z', 'Pt_Lep', 'Eta_Lep', 'Pt_J1', 'Eta_J1',
          'Pt_J2', 'Eta_J2', 'Pt_W', 'Eta_W', 'Pt_top', 'Eta_top', 'mW_inv', 'mtop_inv',
          'MtW', 'MtT', 'SP_LepB', 'SP_LepQ', 'SP_QB', 'SP_LepNu', 'SP_BNu', 'SP_QNu',
          'Ht', 'S_hat', 'Dphi_LepQ', 'Dphi_LepNu', 'Dphi_Wb', 'DR_LepQ', 'DR_LepNu', 'DR_Wb']

In [10]:
scaler_y = StandardScaler()
scaler_y.fit(np.array(train_raw[y_vars]))
scaler_X = StandardScaler()
scaler_X.fit(np.array(train_raw[X_vars]))

X_train_df = pd.DataFrame(scaler_X.transform(np.array(train_raw[X_vars])), columns=X_vars)
X_val_df = pd.DataFrame(scaler_X.transform(np.array(val_raw[X_vars])), columns=X_vars)
X_test_df = pd.DataFrame(scaler_X.transform(np.array(test_raw[X_vars])), columns=X_vars)

y_train_df = pd.DataFrame(scaler_y.transform(np.array(train_raw[y_vars])), columns=y_vars)
y_val_df = pd.DataFrame(scaler_y.transform(np.array(val_raw[y_vars])), columns=y_vars)
y_test_df = pd.DataFrame(scaler_y.transform(np.array(test_raw[y_vars])), columns=y_vars)

In [11]:
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(
        self,
        input_values,
        labels,
        transforms=None,
    ):
        self.input_values = torch.from_numpy(input_values).type(torch.FloatTensor)
        self.labels = torch.from_numpy(labels).type(torch.FloatTensor)
        self.transforms = transforms

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if self.transforms:
            input_value = self.transforms(self.input_values[idx])
        else:
            input_value = self.input_values[idx]
        label = self.labels[idx]
        return input_value, label

In [12]:
train_set = CustomDataset(X_train_df.values, y_train_df.values)
val_set = CustomDataset(X_val_df.values, y_val_df.values)
test_set = CustomDataset(X_test_df.values, y_test_df.values)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=1024, shuffle=True, num_workers = 4
)
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1024, shuffle=False, num_workers = 4
)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=1024, shuffle=False, num_workers = 4
)

In [13]:
import sys
sys.path.append('../src/')
import torch.nn as nn
import torch.nn.functional as F
import sc_models
from torchmetrics.regression import MeanSquaredError, MeanAbsoluteError

In [14]:
class LModel_NF(L.LightningModule):
    def __init__(
            self, model, lr=1e-3,
        ):
        super().__init__()
        self.model = model
        self.criterion = nn.L1Loss()
        self.train_acc = MeanAbsoluteError()
        self.valid_acc = MeanAbsoluteError()
        self.test_acc = MeanAbsoluteError()
        self.lr = lr

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr = self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, min_lr = 1e-5)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch",
                "monitor": "loss/val", 
            },
        }
    def training_step(self, batch, batch_idx):
        x, y = batch
        out = -self.model.log_prob(inputs=y, context=x)
        loss = -self.model.log_prob(inputs=y, context=x).mean()
        self.log("loss/train", loss.detach().item(), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = -self.model.log_prob(inputs=y, context=x)
        loss = -self.model.log_prob(inputs=y, context=x).mean()
        self.log("loss/val", loss.detach().item())

In [15]:
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
import warnings
warnings.filterwarnings("ignore")

def create_trainer(name, max_epochs = 10):
    torch.set_float32_matmul_precision('medium')
    trainer = L.Trainer(
        max_epochs=max_epochs,
        num_sanity_val_steps=1,
        log_every_n_steps=10,
        enable_checkpointing = False,
        enable_model_summary = False,
        enable_progress_bar = False,
        callbacks=[EarlyStopping(monitor="loss/val", mode="min", patience = 2)],
    )
    return trainer

In [16]:
import sc_misc
Cos_lep_light_true, _ = sc_misc.calculate_cosine_theory(val_raw_DM[X_vars], val_raw_DM[y_vars])

In [ ]:
import optuna
from scipy.stats import chisquare

X_val_DM = pd.DataFrame(scaler_X.transform(np.array(val_raw_DM[X_vars])), columns=X_vars)
mean_absolute_error = MeanAbsoluteError()
def objective(trial):
    _context_size = trial.suggest_int('context_size', 8, 64)
    _num_layers = trial.suggest_int('num_layers', 3, 7)
    _spline_layer = trial.suggest_int('spline_layer', 32, 256)
    _spline_num_layers = trial.suggest_int('spline_num_layers', 2, 5)
    
    nu_flow = sc_models.Nu_flow(encoder = sc_models.DeepSet(len(X_vars), _context_size), target_size = len(y_vars), 
                            masking_order = [1, 1, -1, 1, 1, -1],  # masking is done for (x,y) <-> z for neutrino and mediator
                            num_layers = _num_layers, context_size = _context_size, spline_conf = (_spline_layer, _spline_num_layers, 0.1))
    
    pl_model_nu_flow = LModel_NF(nu_flow)
    trainer = create_trainer("nu_flow", max_epochs = 5)
    trainer.fit(
        model=pl_model_nu_flow,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )
    
    with torch.no_grad():
        nu_flow = nu_flow.to(device)
        pred_nuflows, _ = torch.median(nu_flow.sample(num_samples = 5, context = torch.tensor(X_val_DM.values, dtype = torch.float32).to(device)), 1)
        pred_nuflows = scaler_y.inverse_transform(pred_nuflows.cpu())
    Cos_lep_light_mixed_DM, _ = sc_misc.calculate_cosine(pred_nuflows, val_raw_DM[X_vars])
    pred_hist = np.histogram(Cos_lep_light_mixed_DM, bins = 50)[0]
    true_hist = np.histogram(Cos_lep_light_true, bins = 50)[0]
                                    
    return chisquare(pred_hist, true_hist)[0]

study = optuna.create_study()
study.optimize(objective, n_trials = 50)

[I 2024-11-13 17:49:37,616] A new study created in memory with name: no-name-23621fb3-50f3-40e1-88da-8f3601f683f4
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=5` reached.
[I 2024-11-13 17:55:40,001] Trial 0 finished with value: 419.9152697469213 and parameters: {'context_size': 35, 'num_layers': 3, 'spline_layer': 109, 'spline_num_layers': 5}. Best is trial 0 with value: 419.9152697469213.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=5` reached.
[I 2024-11-13 18:06:33,113] Trial 1 finished with value: 732.5860076915831 and parameters: {'context_size': 24, 'num_layers': 6, 'spline_layer': 252, 'spline_num_layers': 3}. Best is trial 0 with value: 419.9152697469213.
GPU available: True (cuda), used

In [ ]:
study.best_params

In [ ]:
trials_df = study.trials_dataframe()
trials_df

In [ ]:
trials_df.to_csv("tuning.csv")